In [1]:
import pickle
import pandas as pd
import numpy as np
from IPython.display import Audio
import torch
import torchaudio
from IPython.display import Audio, display
import json
import os
from pathlib import Path

In [2]:
import pickle
import pandas as pd
import numpy as np
from IPython.display import Audio
import torch
import torchaudio
from IPython.display import Audio, display
import json
import os
from pathlib import Path
# import shutil
from glob import glob



def get_df(exp_root, dir):
    exp_dir = Path(exp_root)/dir
    pkl_results_file = os.path.join(exp_dir, "measures.pickle")

    with open(pkl_results_file, "rb") as handle:
        dfme = pd.read_pickle(handle).reset_index(drop=True)
    return dfme


def get_df_storm(pkl_results_file, snr=None):
    # pkl_results_file = "/data/ephraim/datasets/known_noise/sgmse/exp_l/enhanced/storm_vbd/measures_storm.pickle"
    # pkl_results_file = "/data/ephraim/datasets/known_noise/sgmse/exp_k/enhanced/storm_TIMITChime3/measures_storm.pickle"
        # pkl_results_file = "/data/ephraim/datasets/known_noise/sgmse/exp_l/enhanced/sgmse_WSJ0Chime3/measures_storm.pickle"
    # pkl_results_file = "/data/ephraim/datasets/known_noise/sgmse/exp_k/enhanced/sgmse_TIMITChime3/measures_storm.pickle"
    with open(pkl_results_file, "rb") as handle:
        df2 = pd.read_pickle(handle)
    for i in range(len(df2["snr"])):
        df2.at[i, "snr"] = df2["snr"][i].split("_")[0]
    if snr is None:
        df_storm_ = df2.reset_index(drop=True).sort_values(by=['dir'])
    else:
        df_storm_ = df2[df2["snr"]==snr].reset_index(drop=True).sort_values(by=['dir'])
    return df_storm_


def write_results(df, name_alg,analysis_root,noises):
    print("write")
    # df.to_excel(os.path.join(analysis_root, f"{name_alg}_all.xlsx"))
    # for noisetype in noises:
    #     noise_mine  = df[df["noise_type"]==noisetype]
    #     noise_mine.to_excel(os.path.join(analysis_root, f"{name_alg}_{noisetype}.xlsx"))


def drop_rows_without_comparison(df, ours_df):
    for i in range(len(df["dir"])):
        dir_ = df["dir"][i]
        noisetype = df["noise_type"][i]
        mine_parallel = ours_df[(ours_df["dir"]==dir_) & (ours_df["noise_type"]==noisetype)]
        if len(mine_parallel)==0:
            idx = df[(df.dir==dir_)&(df.noise_type==noisetype)].index
            df = df.drop(idx)
        return df


def get_stats_df(df, alg_name, dir_=None, noise_type=None, snr_=None):
    if dir_:
        df = df[df["dir"]==dir_]
    if noise_type:
        df = df[df["noise_type"]==noise_type]
    if snr_:
        df = df[df["snr"]==snr_]
    dfstats = df.describe()[1:3]
    dfstats = dfstats.assign(alg=alg_name)
    return dfstats


def create_mine_df(exp_root,df_noisy,mine,noises,cols,names,snrs):
    for d in names: 
        dfme = get_df(exp_root,d)
        if dfme is None:
            continue
        dfme = dfme.assign(dir=d)
        cur_df_noisy = dfme[dfme["name"]=="noisy"].reset_index(drop=True)
        cur_df_noisy.loc[0,"dir"] = d
        if df_noisy is None:
            df_noisy = cur_df_noisy
        else:
            df_noisy = pd.concat([df_noisy, cur_df_noisy])
        dfme = dfme[dfme["name"] != "noisy"]
        
        dfme = dfme[dfme["stoi"]>0.5]

        for noisetype in noises :
            for c_snr in snrs:
                c_snr=str(c_snr)
                dfme_cur = dfme[dfme["snr"]==c_snr]
                cur_mine = dfme_cur[dfme_cur["noise_type"]==noisetype]
                cur_mine = cur_mine[cur_mine["pesq"] ==cur_mine["pesq"].max()].reset_index(drop=True)[cols]
                if mine is None:
                    mine = cur_mine
                else:
                    mine = pd.concat([mine, cur_mine])
    mine=mine.reset_index(drop=True)
    return mine, df_noisy

# def copy_wavs(df,wavdst,algname):
#     df = df.reset_index(drop=True)
#     for i in df.index.values:
#         wavpath = df["filename"][i]
#         dir_ = df["dir"][i]
#         snr = df["snr"][i]
#         noisetype = df["noise_type"][i]
#         name = f"{dir_}_{noisetype}_snr{snr}_{algname}.wav"
#         dst = wavdst/f"{name}"
#         print("origin:", wavpath)
#         print("dst:", dst)
#         shutil.copyfile(wavpath, dst)


from audio_tools2  import *
def calc_vad(f, verbose=False):
    test_file=f
    fs,s = read_wav(test_file)
    win_len = int(fs*0.025)
    hop_len = int(fs*0.010)
    sframes = enframe(s,win_len,hop_len) # rows: frame index, cols: each frame
    if verbose:
        plot_this(compute_log_nrg(sframes))

    # percent_high_nrg is the VAD context ratio. It helps smooth the
    # output VAD decisions. Higher values are more strict.
    percent_high_nrg = 0.5

    vad = nrg_vad(sframes,percent_high_nrg)

    if verbose:
        plot_these(deframe(vad,win_len,hop_len),s)
    return deframe(vad,win_len,hop_len) 


   



In [7]:
utterance = "18"

In [8]:
net1_path = "/data/ephraim/datasets/known_noise/undiff_exps/exp_o2_net3_6/"
def get_net(path_):
    cols =["name","stoi","input_stoi","pesq","input_pesq","OVRL","SIG","BAK","si_sdr",'sdr', "sar",'sir',"noise_type", "filename","snr"]
    df3 = get_df(path_,utterance)[cols]
    df3[df3["pesq"] == df3["pesq"].max()]
    net1 = df3[df3["pesq"] == df3["pesq"].max()][["stoi","pesq","input_stoi","input_pesq"]]
    net1["name"] = path_.split("/")[-2]
    return net1,df3
    

In [9]:
net2_path = "/data/ephraim/datasets/known_noise/undiff_exps/exp_o2x_net18/"
net3_path = "/data/ephraim/datasets/known_noise/undiff_exps/exp_o2x_net3_6/"
net4_path = "/data/ephraim/datasets/known_noise/undiff_exps/exp_o2x_net16/"
net5_path = "/data/ephraim/datasets/known_noise/undiff_exps/exp_o2_net18/"
net1,df3 = get_net(net1_path)
net2,df4 = get_net(net2_path)
net3,df5 = get_net(net3_path)
net4,df6 = get_net(net4_path)
net5,df7 = get_net(net5_path)


In [16]:
df7

,name,stoi,input_stoi,pesq,input_pesq,OVRL,SIG,BAK,si_sdr,sdr,sar,sir,noise_type,filename,snr
0,s0.001,0.341655,0.832526,1.042771,1.101931,2.804258,3.190873,3.864371,-31.231243,-18.212776,-18.212776,inf,18,/data/ephraim/datasets/known_noise/undiff_exps...,5
1,noisy,0.832526,0.832526,1.101931,1.101931,1.842682,3.111013,1.831799,4.985583,5.070568,5.070568,inf,18,/data/ephraim/datasets/known_noise/undiff_exps...,5


In [10]:
storm_enhanced_path = "/data/ephraim/datasets/known_noise/undiff_exps/exp_o2x_net3_6/storm/enhanced"
storm_results_path = os.path.join(storm_enhanced_path,"sgmse_TIMITChime3/measures_storm.pickle")
sgmse_results_path = os.path.join(storm_enhanced_path,"sgmse_WSJ0Chime3/measures_storm.pickle")
df_storm = get_df_storm(storm_results_path)
df_sg = get_df_storm(sgmse_results_path)

In [11]:
df_sg[df_sg["noise_type"]==utterance][["name","stoi","input_stoi","pesq","input_pesq","OVRL","SIG","BAK","si_sdr",'sdr', "sar",'sir',"noise_type", "filename","snr"]]

,name,stoi,input_stoi,pesq,input_pesq,OVRL,SIG,BAK,si_sdr,sdr,sar,sir,noise_type,filename,snr
0,_noise18_LJ050-0247_snr5_fileid_18.wav,0.926697,0.832526,1.764428,1.101931,3.093521,3.503463,3.72414,13.556447,14.17912,14.17912,inf,18,/data/ephraim/datasets/known_noise/undiff_exps...,5


In [12]:
baseline1 = df_sg[df_sg["noise_type"]==utterance][["stoi","pesq","input_stoi","input_pesq"]]

In [13]:
df_storm[df_storm["noise_type"]==utterance][["name","stoi","input_stoi","pesq","input_pesq","OVRL","SIG","BAK","si_sdr",'sdr', "sar",'sir',"noise_type", "filename","snr"]]

,name,stoi,input_stoi,pesq,input_pesq,OVRL,SIG,BAK,si_sdr,sdr,sar,sir,noise_type,filename,snr
0,_noise18_LJ050-0247_snr5_fileid_18.wav,0.926606,0.832526,1.720035,1.101931,2.929764,3.275559,3.800985,14.201313,15.005592,15.005592,inf,18,/data/ephraim/datasets/known_noise/undiff_exps...,5


In [14]:
baseline2 = df_storm[df_storm["noise_type"]==utterance][["stoi","pesq","input_stoi","input_pesq"]]

In [15]:
print(utterance)
pd.concat([net1,net2,net3, net4,net5,baseline1,baseline2],axis=0)#.to_excel("/data/ephraim/datasets/known_noise/undiff_exps/exp_o2_net18/storm/enhanced/sgmse_TIMITChime3/baseline.xlsx")

18


,stoi,pesq,input_stoi,input_pesq,name
3,0.923903,1.624119,0.832526,1.101931,exp_o2_net3_6
2,0.924128,1.564895,0.832526,1.101931,exp_o2x_net18
1,0.916542,1.616301,0.832526,1.101931,exp_o2x_net3_6
3,0.918787,1.620616,0.832526,1.101931,exp_o2x_net16
1,0.832526,1.101931,0.832526,1.101931,exp_o2_net18
0,0.926697,1.764428,0.832526,1.101931,NaN
0,0.926606,1.720035,0.832526,1.101931,NaN


In [21]:
print(utterance)
pd.concat([net1,net2,net3,net4,baseline1,baseline2],axis=0)#.to_excel("/data/ephraim/datasets/known_noise/undiff_exps/exp_o2_net18/storm/enhanced/sgmse_TIMITChime3/baseline.xlsx")

18


,stoi,pesq,input_stoi,input_pesq,name
3,0.923903,1.624119,0.832526,1.101931,exp_o2_net3_6
2,0.924128,1.564895,0.832526,1.101931,exp_o2x_net18
1,0.916542,1.616301,0.832526,1.101931,exp_o2x_net3_6
3,0.918787,1.620616,0.832526,1.101931,exp_o2x_net16
0,0.926697,1.764428,0.832526,1.101931,NaN
0,0.926606,1.720035,0.832526,1.101931,NaN
